<img src="images/Picture0.png" width=200x />

# Notebook 05 - Nonlinear Programming Problems and Multiple Objectives

### Covered in this notebook:
- Adding nonlinear constraints to Gurobipy models
- Using multiple objectives in Gurobipy

### Prerequisites:
- N02, N04

### Credits:
- Gurobipy documentation
- https://www.kaggle.com/datasets/arnavvvvv/pokemon-pokedex

We now understand the basics of the Gurobi optimizer tool.  Now, let's move on to some of its finer points.

## Nonlinear programming

Thus far, all of the mathematical programs we have explored have been linear.  That is, the highest power of $x$ in any of our constraint or objective equations has been $1$.  Nonlinear constraints are often far more computationally demanding; where the algorithmically straightforward simplex method is generally sufficient to solve an LP problem, NLP programs require more intensive methods.  A key preoccupation for us, therefore, will be to increase the efficiency of our code.

### Built-in general constraints

We are able to define constraint functions using the `addConstr` and `addConstrs` methods we've already seen by articulating an expression using overloaded operators:

In [1]:
import gurobipy as gp
from gurobipy import GRB

In [2]:
# test model:

n = gp.Model()
x = n.addVar()
y = n.addVar()
z = n.addVar()
c = n.addVar()

Set parameter Username


In [3]:
# x*y <= z
n.addConstr(x*y <= z)

# x^2 == 1
n.addConstr(x**2 == 1)

<gurobi.QConstr Not Yet Added>

In [4]:
# try the following: x*y*z <= 1


However, the `addConstr`/`addConstrs` methods restrict us to only linear or quadratic constraint types.  In addition to this, Gurobi provides built-in constraint methods for a wide range of <strong>general constraint</strong> functions, including polynomials, logarithms, exponential functions, ec.  These built-in methods transform a nonlinear constraint into a collection of fundamental constraints during presolve.  The transformations may be exact reformations or approximations, the granularity of which can be controlled by altering the model environment.

The general constraints handled by Gurobi are documented [here](https://www.gurobi.com/documentation/current/refman/py_model_agc_xxx.html), each named as `Model.addGenConstrXxx()`, where "Xxx" represents the type of function.  While exploring this page, take particular note of the usefulness of the `addGenConstrIndicator` constraint, which allows the user to set a constraint based on the truth of an indicator variable, thus permitting if/then structures.

It should be noted, however, that general function constraints are not Gurobi's strength.  The breadth of optimization applications in the real world deal with discrete data: production is discrete; people are discrete; even money is discrete, when you think about it.  Gurobi is designed with these practical applications in mind, and so we will see some limitations when using general constraints.

Notice in particular that each built-in general constraint defines an equation, with no option for inequalities.  This is not as limiting as it might appear, but requires some thought: we will be required to build complex functions from successive variables and constraints to effect our desired equation when all is said and done.  Try building some general function constraints below.

In [6]:
# e ^ y = z
n.addGenConstrExp(y, z)

<gurobi.GenConstr *Awaiting Model Update*>

In [7]:
# c = max(x, y, z, 1)
n.addGenConstrMax(c, [x, y, z], 1)

<gurobi.GenConstr *Awaiting Model Update*>

In [8]:
# x + y = 1 / (1+exp(-z))
d = n.addVar()
n.update()
n.addConstr(d == x + y)
n.addGenConstrLogistic(z, d)

<gurobi.GenConstr *Awaiting Model Update*>

In [9]:
# x / (1 + y)^2 <= 16
e = n.addVar()
n.addGenConstrPoly(e, y, [1, 1])
f = n.addVar()
n.addGenConstrPow(e, f, -2)
n.addConstr(x*f <= 16)

<gurobi.QConstr Not Yet Added>

### Nonlinear objectives

Gurobi has treatments for <strong>linear</strong> objectives and <strong>quadratic</strong> objectives.  (General nonlinear objectives are not permitted!)  These can be set using the `Model.setObjective()` method with which we are already familiar.

## Multiple objectives

It is not uncommon in large optimization problems to juggle a variety of priorities.  You may want to minimize one element of your model while maximizing another.  Consider our earlier example treating power plants; in a more complex system, we may wish to maximize output while minimizing, say, labor requirements, or environmental cost.

One approach is to frame competing obligations as constraints, but Gurobi allows us more flexibility.  We are able to write multiple objective functions and define their relative importance.  Gurobi ranks objectives through a <strong>hierarchical</strong> approach, a <strong>blended</strong> approach, or a combination of the two.

### Hierarchical objectives

Hierarchical objectives are treated one at a time.  The solver optimizes according to the first objective, which gives it a set of solutions.  Then, it optimizes according to the second objective within that set.  So on it goes down the list.

The objectives are treated in order of their `priority`.  Priority is integral and larger values indicate higher priority.  Objectives' priorities need not be sequential.  So, if three objectives had priorities 9, 1, and 100, the solver would optimize according to the third, then the first, then the second.

If two objectives have the same priority, they are treated simultaneously, using the <strong>blended</strong> approach.

### Blended objectives

A blended approach takes the linear combination of the objectives involved, scaled by the `weight` of each objective.  For example, given the following objectives and weights…

$x + 2y + z$, weight = $1$

$2x + 3z$, weight = $-3$

…the solver would create the following blended objective:

$(1)(x + 2y + z) + (-3)(2x + 3z) = -5x + 2y -8z$

Best practices discourage using weights which are very large or very small (larger than $10^6$ or smaller than $10^{-6}$), as very large coefficients can cause numerical difficulties and very small coefficients may cause values to fall below the solver tolerances and be ignored.

#### Note: Multiple-objective degradation

When multiple objectives are at play, the relative importance of one to another can be difficult to precisely define.   We may find that the "best" solution from a human perspective is not the one which optimizes our first objective; instead, we may wish to slightly reduce optimality of one objective to greatly improve optimality of another.  With the Gurobi tool, this is known as <strong>degradation</strong>: slightly compromising on our earlier objectives to expand the pool of our solutions.  We will not implement degradation at present as it involves altering the parameters of the model, but once that topic is understood, multiple-objective degradation is quite simple to perform.

### Setting multiple objectives

We define multiple objectives using the method `Model.setObjectiveN()`, which takes the following arguments:

* `expr`: the <strong>expression</strong>
    * Note: when multiple objectives are set, all of the objectives must be linear.
* `index`: the <strong>index</strong>.  Required argument; each objective's index must be a unique integer.
    * Not to be confused with `priority`.
* `priority=0`: the objective's <strong>priority</strong>.  Corresponds to the `ObjNPriority` attribute of the objective
* `weight=1`: the objective's <strong>weight</strong>.  Corresponds to the `ObjNWeight` attribute of the objective
* `abstol=1e-6`: the objective's <strong>absolute tolerance</strong>.  Used in multiple-objective degradation.
* `reltol=0`: the objective's <strong>relative tolerance</strong>.  Used in multiple-objective degradation.
* `name=""`: an ASCII string representing the objective's <strong>name</strong>

One thing we notice is that unlike `setObjective`, `setObjectiveN` does not take an optimization `sense`.  Rather, the `weight` of an objective defines whether it is maximized or minimized.  Recall that the default optimization sense of a model is minimize; then to maximize an objective, we give it a negative weight.

Let's implement the above using the following example.

## Exercise: Pokésquad

The file `pokedex.csv` from the [Kaggle public data sets](https://www.kaggle.com/datasets/arnavvvvv/pokemon-pokedex) contains the names, types, and stats of the first generation of Pokemon.  You want to put together a team of exactly 3 pokemon with desirable HP, Attack, Defense, Special Attack, and Special Defense.  Choose how you prioritize your criteria, and create a Gurobi model to deliver your optimal team.

In [10]:
# read in csv:
import pandas as pd

pokedex = pd.read_csv('data/N06_data/pokedex.csv').set_index('Name').T.to_dict('list')

In [11]:
p = gp.Model('Pokesquad')

In [12]:
# variables: all pokemon available; binary
status = p.addVars(len(pokedex), vtype=GRB.BINARY, name=pokedex.keys())

# constraints:
# 3 pokemon in team
p.addConstr(status.sum() == 3)

<gurobi.Constr *Awaiting Model Update*>

In [13]:
p.update()

In [14]:
hp = gp.quicksum(status[i] * pokedex[status[i].varName][3] for i in status)
attack = gp.quicksum(status[i] * pokedex[status[i].varName][4] for i in status)
defense = gp.quicksum(status[i] * pokedex[status[i].varName][5] for i in status)
spAttack = gp.quicksum(status[i] * pokedex[status[i].varName][6] for i in status)
spDefense = gp.quicksum(status[i] * pokedex[status[i].varName][7] for i in status)

p.setObjectiveN(hp, index=0, priority=1, weight=-5, name='HP')
p.setObjectiveN(attack, index=1, priority=1, weight=-8, name='attack')
p.setObjectiveN(defense, index=2, priority=1, weight=-5, name='defense')
p.setObjectiveN(spAttack, index=3, priority=1, weight=-1, name='spAttack')
p.setObjectiveN(spDefense, index=4, priority=1, weight=-1, name='spDefense')

In [15]:
p.optimize()

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[arm] - Darwin 23.5.0 23F79)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1 rows, 202 columns and 202 nonzeros
Model fingerprint: 0x553e0adf
Variable types: 0 continuous, 202 integer (202 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e+00, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+00, 3e+00]

---------------------------------------------------------------------------
Multi-objectives: starting optimization with 5 objectives (1 combined) ...
---------------------------------------------------------------------------
---------------------------------------------------------------------------

Multi-objectives: optimize objective 1 (weighted) ...
---------------------------------------------------------------------------

Optimize a model with 1 rows, 202 columns and 202 nonzeros
Model fingerprint: 

In [16]:
for v in p.getVars():
    if v.x != 0:
        print(v.varName)
        print('    Stats:', pokedex[v.varName])

Gyarados Mega Gyarados
    Stats: ['Water', 'Dark', 640, 95, 155, 109, 70, 130, 81, 'images/176.png', 130]
Mewtwo Mega Mewtwo X
    Stats: ['Psychic', 'Fighting', 780, 106, 190, 100, 154, 100, 130, 'images/202.png', 150]
Mewtwo Mega Mewtwo Y
    Stats: ['Psychic', nan, 780, 106, 150, 70, 194, 120, 140, 'images/203.png', 150]
